In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

In [14]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)


RuntimeError: Could not initialize camera.  Please see error trace.

In [ ]:
model = torchvision.models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)
model.load_state_dict(torch.load("glove_mask_identifier.pth",map_location=torch.device('cpu')))
model.eval()

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
label = widgets.Label(value="This is the initial label text.")

In [ ]:
import torch.nn.functional as F
import time
model.cuda()
display(widgets.HBox([image_widget]))
class_names=['device 1', 'device 2', 'device 3', 'device 4']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def update(change):
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    _, preds = torch.max(y, 1)
    label.value=class_names[preds[0]]
    time.sleep(0.001)

display(label)
update({'new': camera.value})  # we call the function once to intialize

In [ ]:

camera.observe(update, names='value')


In [ ]:
from jetbot import Robot

In [ ]:
robot = Robot()

In [ ]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    #robot.forward(0.4)
    robot.left_motor.value = 0.415
    robot.right_motor.value = 0.4
    #robot.right(0.4)
    #time.sleep(0.5)
    #robot.stop()

def step_backward(change):
    robot.backward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_left(change):
    robot.left(0.15)

def step_right(change):
    robot.right(0.15)

In [ ]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

In [ ]:
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

In [ ]:
camera.unobserve_all()
time.sleep(1.0)
robot.stop()
camera.stop()

In [8]:
from device_predictor import DevicePredictor
camera = Camera.instance(width=224, height=224)
dp = DevicePredictor()


In [14]:
res = dp.predict(camera.value)
print(res)

device 1


In [15]:
camera.stop()